# IBA4315_AS3. Building LLM-powered Applications with APIs (70')
💪 **Objectives:**
- Imagine that you are a developer now. You are asked to build LLM-powered applications by calling API and feeding prompts.
- You are encouraged to improve the applications with proper prompting strategies.
- When you think the applications can perfectly perform the required tasks, you can stop prompt engineering.



## Install Packages
Install all the necessary packages, it may take some time.

In [1]:
# Install required packages
%pip install gradio
%pip install openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.4/312.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 1.0 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 

## Import and Setup
**Please make double sure not to share the api with anyone else.**

In [2]:
# import the packages
from openai import OpenAI
import gradio as gr
import json
from typing import List, Dict, Tuple

## Kindly keep the key for yourself and for the assignment only
API_SECRET_KEY = "sk-nWSoC3jZh6LvMSVe30541cAf25B447D9Bf1fFc19239cDcA7"
BASE_URL = "https://chatapi.onechat.fun/v1"
client = OpenAI(api_key=API_SECRET_KEY, base_url=BASE_URL)

# Check if you have set your ChatGPT API successfully
# You should see "Set ChatGPT API sucessfully!!" if nothing goes wrong.
try:
    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages = [{'role':'user','content': "test"}],
            max_tokens=1,
    )
    print("Set ChatGPT API sucessfully!!")
except:
    print("There seems to be something wrong with your ChatGPT API. Please follow our demonstration in the slide to get a correct one.")

Set ChatGPT API sucessfully!!


## Part 1: Review Analyzer (35')  

In this task, you are asked to prompt your chatbot into a **review analyzer.**  When the resturant inputs any reviews, it can **analyze the reviews for the resturant.**Your prompts should enable the chatbot to:
   
👇***Functional requirements (20')***  
*   Count the number of positive and negative reviews
*   Extract all the recommended dishes in the given reviews
*   Summarize any negative aspects that are mentioned in the review
*   Identify the ID of any negative reviews that requires the resturant's immediate reply and generate a reply for the resturant (using right language, no more than 100 words)  

👇***Non-functional requirements (15')***  
*   The output should be as short as possible, well organized in a JSON object
*   Properly deal with the cases where the relevant information is not present in the given review
*   Prevent the users from using the review analyzer for any irrelevant tasks



In [5]:
## TODO: Design the prompt in English by defining prompt_for_review_analyzer ""
## TODO: Design the prompt in English by defining prompt_for_review_analyzer ""
prompt_for_review_analyzer = f"""
Given the following reviews, please:

1. Count the number of positive and negative reviews.
2. Extract all the recommended dishes in the given reviews.
3. Summarize any negative aspects that are mentioned.
4. Identify the ID of any negative reviews that requires the restaurant's immediate reply
5. generate a complete reply for the restaurant in the language consistent with the negative review.

For example:
i. Positive reviews: ; Negative reviews:
ii. recommended dishes:
iii. negative aspects:
iv: ID of any negative reviews:
V. Repley:

Please remeber that the output should be short and well organized in a JSON object.
If the relevant information is not present in the given review, use null.
If the users is using the review analyzer for any irrelevant tasks, please provide a warning message to the user.
"""


# Function to reset the conversation
def reset() -> List:
    return []

# Function to call the model to generate
def interact_review_analyzer(prompt: str, reviews: str, temp = 1.0) -> List[Tuple[str, str]]:
    '''
    * Arguments

      - prompt: the prompt that we use in this section

      - reviews: the reviews to be analyzed

      - temp: the temperature parameter of this model. Temperature is used to control the output of the chatbot.
              The higher the temperature is, the more creative response you will get.

    '''
    input = f"{prompt}\n{reviews}"
    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages = [{'role':'user','content': input}],
            temperature = temp,
            max_tokens=200,
    )

    return [(input, response.choices[0].message.content)]

# Function to export the whole conversation
def export_review_analyzer(chatbot: List[Tuple[str, str]], reviews: str) -> None:
    '''
    * Arguments

      - chatbot: the model itself, the conversation is stored in list of tuples

      - reviews: the reviews to be analyzed

    '''
    target = {"chatbot": chatbot, "reviews": reviews}
    with open("Part1.json", "w") as file:
        json.dump(target, file)

# This part generates the Gradio UI interface
with gr.Blocks() as demo:
    gr.Markdown("# Part1: Review_analyzer\nFill in any reviews and let the chatbot analyze it for you!!")
    chatbot = gr.Chatbot()
    prompt_textbox = gr.Textbox(label="Prompt", value=prompt_for_review_analyzer, visible=False)
    review_textbox = gr.Textbox(label="Reviews", interactive = True, value = f"""####
1. 清真手抓羊肉饭，看起来不错，但是吃到嘴里发现这里面水分太多，\
基本没有羊肉什么事，妥妥成为胡萝卜拌饭，汤也咸，说明他们放的羊肉不够，\
羊油不够后以水代之来煮，使得整个手抓饭就软绵绵一团甜米饭，希望日后能改一改，日后再见。\n
####
2. 作为港中深的下园食堂，入驻的商家蛮多的，菜也普遍还不错，完胜高中食堂。\
本人比较喜欢重庆小面窗口的重庆小面和燃面，口味香辣，吃起来很爽。\
麻辣烫窗口也不错，可以自助选菜，自助加麻汁、醋等小料调味。\
自选菜窗口价格较贵,不推荐。书亦烧仙草的茉香奶绿真的好喝！\n
####
3. 雨田粿条太喜欢啦！份量足，新鲜美味，大骨汤现场在熬着，喝起来很舒服，有味！\
猪杂是鲜肉，猪心，粉肠，肉饼等，丰富，两片菜叶子也新鲜。价格也实惠，一份17\
也有牛肉可选，酱料也有沙茶酱哇！卫生健康之选。用餐很愉快，保持喔""")
    with gr.Column():
        gr.Markdown("#  Temperature\n Temperature is used to control the output of the chatbot. The higher the temperature is, the more creative response you will get.")
        temperature_slider = gr.Slider(0.0, 2.0, 1.0, step = 0.1, label="Temperature")
    with gr.Row():
        sent_button = gr.Button(value="Send")
        reset_button = gr.Button(value="Reset")

    with gr.Column():
        gr.Markdown("#  Save your Result.\n After you get a satisfied result. Click the export button to recode it.")
        export_button = gr.Button(value="Export")
    sent_button.click(interact_review_analyzer, inputs=[prompt_textbox, review_textbox, temperature_slider], outputs=[chatbot])
    reset_button.click(reset, outputs=[chatbot])
    export_button.click(export_review_analyzer, inputs=[chatbot, review_textbox])


demo.launch(debug = True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://f1011e447386f77021.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f1011e447386f77021.gradio.live


## Part 2: IBA4315_bot (35')
In Part 2, you are asked to prompt a chatbot for the course IBA4315. Interact with the bot for **3 rounds** by typing (ir)relevant questions in the Input field to **demonstrate that your bot is capable of** answering common questions related to course syllabus, assignment deadlines, quiz time, grading components, general policy. **Indicate any principles you considered when designing and refining your prompts.**


You need to:  
1. Come up with a prompt for **IBA4315_bot** and fill it in **prompt_for_faqbot**.
`👇The course information you may need is provided in the code cell.`
1. **Hit the run button.** An interface wiill pop up.
2. **Click the public or local URL** and open the chatbot in a seperate window of your browser. Type your question in the Input field.
3. Hit the "Send" button to produce the results. (You can use the "Temperature" slide to control the creativeness of the output.)
4. If you **want to change your prompt**, hit the run button again to stop the cell. Then go back to step 1.
5. **After you get the desired result**, create the **screen shot in the seperate window** and hit the button **"Export"** to save your result. There will be a file named **part2.json** appearing in the file list.

**❗Important Note:❗**

*  **If you hit the "Export" button again, the previous result will be covered.**
*  **You should keep in mind that even with the exact same prompt, the output might still differ.**
*  **Remember to stop this cell before you go on to the next one.**

### Please indicate what principles or techniques you incorporate in `prompt_for_faqbot`?

In [6]:
# TODO: Design the prompt in English by defining prompt_for_faqbot""
chatbot_task = "Answer common questions about course syllabus, \
assignment deadlines, quiz time, grading components, general policy of IBA4315"

prompt_for_faqbot = f"""

You are a smart course assistant, please be polite and interactive. You should be able to answer any questions about the IBA4315 course, such as:

- What is the grading component of the course?
- When are the assignment deadlines?
- When are the quiz times?
- What is the general policy of the course?
- Tell me the syllabus of the course?


The information you may refer to is shown as follow:
##Grading Component##
Participation - 10%
Assignment (4 coding and 1 case analysis) - 38%
In-class quiz (3 times) - 12%
Term project (40%)

##General Policy##
Use of AI Policy: Directly coping GPT-generated answers is not allowed (plagiarism)
No Late Policy: For each type of submissions (i.e., assignments and project),\
you can have a one-day extension without penalty, but only one time for each
Penalty is 20% deduction of your grade for each additional late day.
Collaboration Policy: Free-rider not allowed!

##syllabus##
Jan 09 & 11	- Course overview and Intro to AI, ML, DL
Jan 16 & 18	- Acquiring data for AI/ML solutions
Jan 23 & 25	- Neural networks (I) - Basics
Jan 30 & Feb 01	- Neural networks (II) - Python Demonstrations & Optimizations
Feb 27 & 29	- Neural networks (III) - MLP&CNN; On-Device and Cloud AI
Mar 05 & 07	- Intro to Gen AI & LLMs; Prompt Engineering (I)
Mar 12 & 14	- Prompt Engineering(II) & OpenAI API
Mar 19 & 21	- Prompt Engineering(III) & LLM Techniques (I)
Mar 26 & 28	- LLM Techniques (II)
Apr 02 & 07	- Image generation model
Apr 09 & 11 - AI Agent and other trending use cases of LLMs
Apr 16 & 18 - Case study: DBS BANK; Managing AI in organizations
Apr 23 & 25 - Invited lecture by guest speaker; Term project presentation 1
Apr 30 - Term project presentation 2

##Important Timeline##
Assignment:
AS1, Noon on Jan 31 2024
AS2, Noon on Mar 06 2024
AS3, Noon on Mar 27 2024
AS4, Noon on Apr 03 2024
Case, Noon on Apr 21 2024
Quiz:
Quiz1, Feb 01 2024
Quiz2, Mar 21 2024
Quiz3, Apr 11 2024
Project:
Project proposal, Noon on Mar 13 2024
Project presentation session 1, Mar 25 2024
Project presentation session 1, Mar 30 2024
Project report, Noon on May 12 2024
"""

# Function to clear the conversation
def reset() -> List:
    return []

# Function to call the model to generate
def interact_customize(chatbot: List[Tuple[str, str]], prompt: str ,user_input: str, temperature = 1.0) -> List[Tuple[str, str]]:
    '''
    * Arguments

      - chatbot: the model itself, the conversation is stored in list of tuples

      - prompt: the prompt for your desginated task

      - user_input: the user input of each round of conversation

      - temp: the temperature parameter of this model. Temperature is used to control the output of the chatbot.
              The higher the temperature is, the more creative response you will get.

    '''
    try:
        messages = []
        messages.append({'role': 'user', 'content': prompt})
        for input_text, response_text in chatbot:
            messages.append({'role': 'user', 'content': input_text})
            messages.append({'role': 'assistant', 'content': response_text})

        messages.append({'role': 'user', 'content': user_input})

        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages = messages,
            temperature = temperature,
            max_tokens=200,
        )

        chatbot.append((user_input, response.choices[0].message.content))

    except Exception as e:
        print(f"Error occurred: {e}")
        chatbot.append((user_input, f"Sorry, an error occurred: {e}"))
    return chatbot

# Function to export the whole conversation log
def export_customized(chatbot: List[Tuple[str, str]], description: str) -> None:
    '''
    * Arguments

      - chatbot: the model itself, the conversation is stored in list of tuples

      - description: the description of this task

    '''
    target = {"chatbot": chatbot, "description": description}
    with open("part2.json", "w") as file:
        json.dump(target, file)

# This part constructs the Gradio UI interface
with gr.Blocks() as demo:
    gr.Markdown("# Part2: IBA4315_bot\nThe chatbot is able to answer common questions about course syllabus, \
    assignment deadlines, quiz time, grading components, general policy of IBA4315. Try to interact with it!!")
    chatbot = gr.Chatbot()
    desc_textbox = gr.Textbox(label="Description of the task", value=chatbot_task, interactive=False)
    prompt_textbox = gr.Textbox(label="Prompt", value=prompt_for_faqbot, visible=False)
    input_textbox = gr.Textbox(label="Input")
    with gr.Column():
        gr.Markdown("#  Temperature\n Temperature is used to control the output of the chatbot. The higher the temperature is, the more creative response you will get.")
        temperature_slider = gr.Slider(0.0, 2.0, 1.0, step = 0.1, label="Temperature")
    with gr.Row():
        sent_button = gr.Button(value="Send")
        reset_button = gr.Button(value="Reset")
    with gr.Column():
        gr.Markdown("#  Save your Result.\n After you get a satisfied result. Click the export button to recode it.")
        export_button = gr.Button(value="Export")
    sent_button.click(interact_customize, inputs=[chatbot, prompt_textbox, input_textbox, temperature_slider], outputs=[chatbot])
    reset_button.click(reset, outputs=[chatbot])
    export_button.click(export_customized, inputs=[chatbot, desc_textbox])

demo.launch(debug = True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://8bbbef0ee840ca588e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8bbbef0ee840ca588e.gradio.live




1. Clarity: clearly define the role and task of the model to guide it towards generating the correct responses.

2. Context: Where possible, the prompt should provide sufficient context to help the model better understand the question.

3. Conciseness: The prompt should be as concise as possible, avoiding unnecessary or irrelevant information.

4. User Engagement: The prompt encourages users to interact with the bot, enhancing user engagement.

5. Avoid negative sentences,try to directly tell the bot what is needed.